# ViT Fine-Tuning Demo

*Based on: https://huggingface.co/blog/fine-tune-vit*

Before running, download the [TrashNet dataset](https://github.com/garythung/trashnet/raw/master/data/dataset-resized.zip) and extract it to the 'dataset' folder.

In [1]:
%pip install huggingface
%pip install datasets
%pip install pillow
%pip install transformers
%pip install scikit-learn
%pip install transformers[torch]
%pip install tensorboardX
# PyTorch + CUDA should be installed manually

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


## Dataset Processing

In [2]:
from datasets import load_dataset
ds = load_dataset("imagefolder", data_dir="/home/schakkera/CSE512/cse512-project/data")
# ds = load_dataset("imagefolder", data_dir="/home/starc/SBU/Sem-3/ML/Project/cse512-project/data")

Resolving data files:   0%|          | 0/40800 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/10000 [00:00<?, ?it/s]

In [3]:
from transformers import ViTImageProcessor, ViTConfig
from torchvision.transforms import Compose, RandomResizedCrop, RandomHorizontalFlip, ToTensor, Normalize, Resize, CenterCrop, ConvertImageDtype, InterpolationMode, ColorJitter
import torch

model_name_or_path = 'google/vit-base-patch16-224'
config = ViTConfig.from_pretrained(model_name_or_path)
processor = ViTImageProcessor.from_pretrained(model_name_or_path)

_train_transform = Compose([
    RandomResizedCrop(config.image_size, interpolation=InterpolationMode.BICUBIC, antialias=None),
    RandomHorizontalFlip(),
    ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    ToTensor(),
    ConvertImageDtype(torch.float),
    Normalize(
        mean=processor.image_mean,
        std=processor.image_std
    )
])

_val_transform = Compose([
    Resize(config.image_size, interpolation=InterpolationMode.BICUBIC, antialias=None),
    CenterCrop(config.image_size),
    ToTensor(),
    ConvertImageDtype(torch.float),
    Normalize(
        mean=processor.image_mean,
        std=processor.image_std
    )
])

def train_transform(example_batch):
    example_batch['pixel_values'] = [
        _train_transform(img.convert('RGB')) for img in example_batch['image']
    ]
    return example_batch

def val_transform(example_batch):
    example_batch['pixel_values'] = [
        _val_transform(img.convert('RGB')) for img in example_batch['image']
    ]
    return example_batch

ds['train'].set_transform(train_transform)
ds['test'].set_transform(val_transform)

## Training

In [4]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [5]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

def ova_compute_metrics(p):
    return metric.compute(predictions=np.argmin(p.predictions, axis=1), references=p.label_ids)

/tmp/ipykernel_13598/596433834.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [6]:
from transformers import ViTForImageClassification

labels = ds['train'].features['label'].names

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    ignore_mismatched_sizes=True,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import TrainingArguments
from transformers.trainer_utils import get_last_checkpoint
import os

if os.path.exists('./vit-base-fmnist'):
    last_checkpoint = get_last_checkpoint("./vit-base-fmnist")
else:
    last_checkpoint = None

training_args = TrainingArguments(
  output_dir="./vit-base-fmnist",
  per_device_train_batch_size=256,
  evaluation_strategy="steps",
  num_train_epochs=32,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=20,
  learning_rate=3e-5, #3e-5
  save_total_limit=1,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
  resume_from_checkpoint=last_checkpoint
)

# if os.path.exists('./vit-base-fmnist-ovareg'):
#     last_checkpoint = get_last_checkpoint("./vit-base-fmnist-ovareg")
# else:
#     last_checkpoint = None

# training_args = TrainingArguments(
#   output_dir="./vit-base-fmnist-ovareg",
#   per_device_train_batch_size=256,
#   evaluation_strategy="steps",
#   num_train_epochs=32,
#   fp16=True,
#   save_steps=100,
#   eval_steps=100,
#   logging_steps=20,
#   learning_rate=3e-5, #3e-5
#   save_total_limit=1,
#   remove_unused_columns=False,
#   push_to_hub=False,
#   report_to='tensorboard',
#   load_best_model_at_end=True,
#   resume_from_checkpoint=last_checkpoint
# )

# if os.path.exists('./vit-base-fmnist-csce'):
#     last_checkpoint = get_last_checkpoint("./vit-base-fmnist-csce")
# else:
#     last_checkpoint = None

# training_args = TrainingArguments(
#   output_dir="./vit-base-fmnist-csce",
#   per_device_train_batch_size=256,
#   evaluation_strategy="steps",
#   num_train_epochs=32,
#   fp16=True,
#   save_steps=100,
#   eval_steps=100,
#   logging_steps=20,
#   learning_rate=3e-5, #3e-5
#   save_total_limit=1,
#   remove_unused_columns=False,
#   push_to_hub=False,
#   report_to='tensorboard',
#   load_best_model_at_end=True,
#   resume_from_checkpoint=last_checkpoint
# )

In [8]:
from transformers import Trainer
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class WeightedCrossEntropyTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.cost_matrix = torch.tensor([[1., 1., 1., 1., 0.3, 1., 1., 1., 1., 0.3], 
                                        [0.33, 1., 0.93, 1., 0.1, 1., 1., 1., 1., 0.1], 
                                        [0.36, 1., 1., 1., 0.1, 1., 1., 1., 1., 0.1], 
                                        [0.33, 1., 0.93, 1., 0.1, 1., 1., 1., 1., 0.1], 
                                        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], 
                                        [0.33, 1., 0.93, 1., 0.1, 1., 1., 1., 1., 0.1], 
                                        [0.33, 1., 0.93, 1., 0.1, 1., 1., 1., 1., 0.1], 
                                        [0.33, 1., 0.93, 1., 0.1, 1., 1., 1., 1., 0.1], 
                                        [0.33, 1., 0.93, 1., 0.1, 1., 1., 1., 1., 0.1], 
                                        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]).to(device)
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        # print(labels.size(), labels)
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        exp_logits = torch.exp(logits)
        batch_spec_weights = self.cost_matrix[labels, :]
        weighted_exp = exp_logits * batch_spec_weights
        weighted_softmax = weighted_exp/torch.sum(weighted_exp, axis=1).reshape(weighted_exp.size(0), -1)
        log_weighted = torch.log(weighted_softmax)
        onehot = F.one_hot(labels, self.cost_matrix.size(0))
        ind_cross_entropy = -log_weighted*onehot
        
        # compute custom loss (suppose one has 3 labels with different weights)
        loss = torch.mean(torch.sum(ind_cross_entropy, axis=1))
        return (loss, outputs) if return_outputs else loss

class OVARegressionTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.cost_matrix = 10*torch.tensor([[0., 3., 2.8, 3., 0.3, 3., 3., 3., 3., 0.3], 
                                        [0.33, 0., 0.93, 1., 0.1, 1., 1., 1., 1., 0.1], 
                                        [0.36, 1.07, 0., 1.07, 0.1, 1.07, 1.07, 1.07, 1.07, 0.1], 
                                        [0.33, 1., 0.93, 0., 0.1, 1., 1., 1., 1., 0.1], 
                                        [1.33, 4., 3, 4., 0., 4., 4., 4., 4., 1.], 
                                        [0.33, 1., 0.93, 1., 0.1, 0., 1., 1., 1., 0.1], 
                                        [0.33, 1., 0.93, 1., 0.1, 1., 0., 1., 1., 0.1], 
                                        [0.33, 1., 0.93, 1., 0.1, 1., 1., 0., 1., 0.1], 
                                        [0.33, 1., 0.93, 1., 0.1, 1., 1., 1., 0., 0.1], 
                                        [1.33, 4., 3, 4., 1., 4., 4., 4., 4., 0.]]).to(device)
    
    
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        # print(labels.size(), labels)
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        onehot = F.one_hot(labels, self.cost_matrix.size(0))
        zval = (onehot*2)-1
        batch_spec_weights = self.cost_matrix[labels, :]
        regress = logits-batch_spec_weights
        # print(np.argmin(regress.detach().numpy(), axis=1), labels)
        zprod = zval*regress
        
        # compute custom loss (suppose one has 3 labels with different weights)
        loss=torch.mean(torch.sum(torch.log(torch.exp(zprod)+1), axis=1))
        # print(loss)
        return (loss, outputs) if return_outputs else loss

In [9]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    tokenizer=processor,
)
# trainer = OVARegressionTrainer(
#     model=model,
#     args=training_args,
#     data_collator=collate_fn,
#     compute_metrics=ova_compute_metrics,
#     train_dataset=ds["train"],
#     eval_dataset=ds["test"],
#     tokenizer=processor,
# )
# trainer = WeightedCrossEntropyTrainer(
#     model=model,
#     args=training_args,
#     data_collator=collate_fn,
#     compute_metrics=compute_metrics,
#     train_dataset=ds["train"],
#     eval_dataset=ds["test"],
#     tokenizer=processor,
# )

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [10]:
train_results = trainer.train(resume_from_checkpoint=last_checkpoint)
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

Step,Training Loss,Validation Loss,Accuracy
100,0.492400,0.478276,0.821000
200,0.398300,0.423103,0.842600
300,0.399500,0.367951,0.860400
400,0.361700,0.297751,0.885900
500,0.337800,0.289476,0.895700
600,0.353100,0.288194,0.892600
700,0.317300,0.252942,0.910100
800,0.308800,0.231247,0.919600
900,0.289500,0.252804,0.910900
1000,0.284700,0.287570,0.897200


***** train metrics *****
  epoch                    =          32.0
  total_flos               = 94231959369GF
  train_loss               =        0.2521
  train_runtime            =    4:12:37.34
  train_samples_per_second =        86.136
  train_steps_per_second   =         0.338


In [11]:
metrics = trainer.evaluate(ds['test'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =       32.0
  eval_accuracy           =     0.9354
  eval_loss               =     0.2015
  eval_runtime            = 0:00:53.51
  eval_samples_per_second =    186.856
  eval_steps_per_second   =     23.357
